# Text Preprocessing Exercise
##### Author: Alex Sherman | alsherman@deloitte.com

#### Agenda

1. SpaCy
2. Text Tokenization, POS Tagging, Parsing, NER
3. Text Pipelines
4. Python Fundamentals: Collections, Itertools, list comprehensions, and Sorted
5. Text Rule-based matching
6. Advanced SpaCy Examples

In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine
from IPython.core.display import display, HTML
from IPython.display import Image
from configparser import ConfigParser, ExtendedInterpolation

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('../../config.ini')
DB_PATH = config['DATABASES']['PROJECT_DB_PATH']

In [ ]:
# confirm DB_PATH is in the correct db directory, otherwise the rest of the code will not work
DB_PATH

In [ ]:
# Code to create a local database - commented out as the database is already provided

# f = r'..\raw_data\pubmed\pubmed_2M.txt'                                     # path to pubmed data (may differ on your machine)
# engine = create_engine(DB_PATH)                                             # DB connection string
# pd.read_csv(f, header=None, names=['text']).to_sql('pubmed', con=engine)    # read dataset and write to db

In [ ]:
# check for the names of the tables in the database
engine = create_engine(DB_PATH)
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", con=engine)

In [ ]:
# read the oracle 10k documents 
df = pd.read_sql(
      "SELECT * FROM pubmed"
    , index_col='index'
    , con=engine
)

df.head()

In [ ]:
# increase the number of characters displayed in each column
pd.set_option('display.max_colwidth', 100)
df.head(10)

In [ ]:
# example text
text = df.text[27]
text

### SpaCy

"SpaCy is a free, open-source library for advanced Natural Language Processing (NLP) in Python.

If you're working with a lot of text, you'll eventually want to know more about it. For example, what's it about? What do the words mean in context? Who is doing what to whom? What companies and products are mentioned? Which texts are similar to each other?

SpaCy is designed specifically for production use and helps you build applications that process and "understand" large volumes of text. It can be used to build information extraction or natural language understanding systems, or to pre-process text for deep learning.

SpaCy is not research software. It's built on the latest research, but it's designed to get things done. This leads to fairly different design decisions than NLTK or CoreNLP, which were created as platforms for teaching and research. The main difference is that SpaCy is integrated and opinionated. SpaCy tries to avoid asking the user to choose between multiple algorithms that deliver equivalent functionality. Keeping the menu small lets SpaCy deliver generally better performance and developer experience."

### SpaCy Features 

NAME |	DESCRIPTION |
:----- |:------|
Tokenization|Segmenting text into words, punctuations marks etc.|
Part-of-speech (POS) Tagging|Assigning word types to tokens, like verb or noun.|
Dependency Parsing|	Assigning syntactic dependency labels, describing the relations between individual tokens, like subject or object.|
Lemmatization|	Assigning the base forms of words. For example, the lemma of "was" is "be", and the lemma of "rats" is "rat".|
Sentence Boundary Detection (SBD)|	Finding and segmenting individual sentences.|
Named Entity Recognition (NER)|	Labelling named "real-world" objects, like persons, companies or locations.|
Similarity|	Comparing words, text spans and documents and how similar they are to each other.|
Text Classification|	Assigning categories or labels to a whole document, or parts of a document.|
Rule-based Matching|	Finding sequences of tokens based on their texts and linguistic annotations, similar to regular expressions.|
Training|	Updating and improving a statistical model's predictions.|
Serialization|	Saving objects to files or byte strings.|

SOURCE: https://spacy.io/usage/spacy-101

## SpaCy Installation:
- Windows: Download Microsoft Visual C++: 
1.	Go to: https://www.visualstudio.com/downloads/#build-tools-for-visual-studio-2017
2.	Download the first link for Visual Studio Community 2017
3.	During the install select the option to install Desktop with Development C++ (see image below)

In [ ]:
# Desktop with Development C++
Image("../../raw_data/images/visual_studio_community.png", width=1000)

##### SpaCy Installation
Run the following using git bash as an administrator (i.e. right click on the git bash logo and select 'Run as Admin')
- conda install -c conda-forge spacy
- python -m spacy download en

##### if you run into an error try the following:
- python -m spacy link en_core_web_sm en
- SOURCE: https://github.com/explosion/spaCy/issues/950

##### Optional to install a convolutional neural network model  (~800MB). This is the model I will use in class:
- python -m spacy download en_core_web_lg

##### Test the following code from git bash (even if previous step failed):
start python
- python -i

test if SpaCy was downloaded
- import spacy

approach 1: test if model downloaded
- nlp = spacy.load('en') 

appraoch 2: test this if spacy.load('en') failed
- import en_core_web_sm
- nlp = en_core_web_sm.load()

exit Python
- exit()


##### Optional - install on an AWS EC2 instance
Instance: Amazon Linux 2 LTS Candidate 2 AMI (HVM), SSD Volume Type

- #!/bin/bash
- sudo yum update -y
- sudo yum groupinstall 'Development Tools' -y
- sudo easy_install pip
- sudo yum install python-devel -y
- sudo pip install spacy
- sudo python -m spacy download en_core_web_lg

In [ ]:
# confirm which conda environment you are using - make sure it is one with SpaCy installed
import sys
sys.executable

# if you have difficulty importing spacy try the following in git bash
# conda install ipykernel --name Python3

In [ ]:
import spacy
from spacy import displacy

In [ ]:
%%time

# read in a simple (small) English language model
nlp = spacy.load('en')

# another approach:
# import en_core_web_sm
# nlp = en_core_web_sm.load()

In [ ]:
%%time 

# read in a (large) convolutional neural network model
# this will only work after the CNN model is downloaded (~800MB)
# e.g. python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg') 

In [ ]:
# instantiate the document text
doc = nlp(text)

In [ ]:
# view the text from the SpaCy object
doc

In [ ]:
# which the SpaCy document methods and attributes
print(dir(doc))

### NLP Pipeline

When you read the text into spaCy, e.g. doc = nlp(text), you are applying a pipeline of nlp processes to the text.
by default spaCy applies a tagger, parser, and ner, but you can choose to add, replace, or remove these steps.
Note: Removing unnecessary steps for a given nlp can lead to substantial descreses in processing time.

In [ ]:
# SpaCy pipeline
spacy_url = 'https://spacy.io/assets/img/pipeline.svg'
iframe = '<iframe src={} width=1000 height=200></iframe>'.format(spacy_url)
HTML(iframe)

### Tokenization

SpaCy first tokenizes the text, i.e. segments it into words, punctuation and so on. This is done by applying rules specific to each language. For example, punctuation at the end of a sentence should be split off – whereas "U.K." should remain one token. 

In [ ]:
tokenization_url = 'https://spacy.io/assets/img/tokenization.svg'
iframe = '<iframe src={} width=650 height=400></iframe>'.format(tokenization_url)
HTML(iframe)

##### Lexeme - entries in the vocabulary

In [ ]:
# import a list of stop words from SpaCy
from spacy.lang.en.stop_words import STOP_WORDS

print('Example stop words: {}'.format(list(STOP_WORDS)[0:10]))

In [ ]:
nlp.vocab['have']

In [ ]:
print(dir(nlp.vocab['have']))

In [ ]:
nlp.vocab['have'].is_stop

In [ ]:
# search for word in the SpaCy vocabulary and
# change the is_stop attribute to True (default is False)

for word in STOP_WORDS:
    nlp.vocab[word].is_stop = True

### Part-of-speech (POS) Tagging

After tokenization, spaCy can parse and tag a given Doc. This is where the statistical model comes in, which enables spaCy to make a prediction of which tag or label most likely applies in this context. A model consists of binary data and is produced by showing a system enough examples for it to make predictions that generalize across the language – for example, a word following "the" in English is most likely a noun.

Annotation | Description
:----- |:------|
Text |The original word text|
Lemma |The base form of the word.|
POS |The simple part-of-speech tag.|
Tag |The detailed part-of-speech tag.|
Dep |Syntactic dependency, i.e. the relation between tokens.|
Shape |The word shape – capitalisation, punctuation, digits.|
Is Alpha |Is the token an alpha character?|
Is Stop |Is the token part of a stop list, i.e. the most common words of the language?|

In [ ]:
# review document
doc

In [ ]:
# check if POS tags were added to the doc in the NLP pipeline
doc.is_tagged

In [ ]:
# print column headers
print('{:15} | {:15} | {:8} | {:8} | {:11} | {:8} | {:8} | {:8} | '.format(
    'TEXT','LEMMA_','POS_','TAG_','DEP_','SHAPE_','IS_ALPHA','IS_STOP'))

# print various SpaCy POS attributes
for token in doc[0:20]:
    print('{:15} | {:15} | {:8} | {:8} | {:11} | {:8} | {:8} | {:8} |'.format(
          token.text, token.lemma_, token.pos_, token.tag_, token.dep_
        , token.shape_, token.is_alpha, token.is_stop))

### Text Dependency Parsing

spaCy features a fast and accurate syntactic dependency parser, and has a rich API for navigating the tree. The parser also powers the sentence boundary detection, and lets you iterate over base noun phrases, or "chunks". You can check whether a Doc  object has been parsed with the doc.is_parsed attribute, which returns a boolean value. If this attribute is False, the default sentence iterator will raise an exception.

In [ ]:
# check is document has been parsed (dependency parsing)
doc.is_parsed

In [ ]:
print('{:15} | {:10} | {:15} | {:10} | {:25} | {:25}'.format(
    'TEXT','DEP','HEAD TEXT','HEAD POS','CHILDREN','LEFTS'))

for token in doc[0:20]:
    print('{:15} | {:10} | {:15} | {:10} | {:25} | {:25}'.format(
        token.text, token.dep_, token.head.text, token.head.pos_,
        str([child for child in token.children]), str([t.text for t in token.lefts])))

#### NOUN CHUNCKS:

| **TERM** | Definition |
|:---|:---:|
| **Text** | The original noun chunk text |
| **Root text** | The original text of the word connecting the noun chunk to the rest of the parse |
| **Root dependency** | Dependency relation connecting the root to its head |
| **Root head text** | The text of the root token's head |

In [ ]:
print('{:15} | {:10} | {:15} | {:40}'.format('ROOT_TEXT','ROOT','DEPENDENCY','TEXT'))

for chunk in list(doc.noun_chunks)[0:20]:
    print('{:15} | {:10} | {:15} | {:40}'.format(
        chunk.root.text, chunk.root.dep_, chunk.root.head.text, chunk.text))

In [ ]:
# dependency visualization

# show visualization in Jupyter Notebook
displacy.render(docs=doc, style='dep', jupyter=True)

# Another Option
# uncomment and run the below code, then open another browser tab and go to http://localhost:5000
# when you are done (before you run the next cell in the notebook) stop this cell
# displacy.serve(docs=doc, style='dep')

### Named Entity Recognition (NER)

A named entity is a "real-world object" that's assigned a name – for example, a person, a country, a product, or a book title. spaCy can recognise various types of named entities in a document, by asking the model for a prediction. 

In [ ]:
ner_text = "When I told John that I wanted to move to Alaska, he warned me that I'd have trouble finding a Starbucks there."
ner_doc = nlp(ner_text)

In [ ]:
print('{:10} | {:15}'.format('LABEL','ENTITY'))

for ent in ner_doc.ents[0:20]:
    print('{:10} | {:50}'.format(ent.label_, ent.text))

In [ ]:
# ent methods and attributes
print(dir(ent))

In [ ]:
# entity visualization
# after you run this code, open another browser and go to http://localhost:5000
# when you are done (before you run the next cell in the notebook) stop this cell 

displacy.render(docs=ner_doc, style='ent', jupyter=True)

# Pipeline

If you have a sequence of documents to process, you should use the Language.pipe()  method. The method takes an iterator of texts, and accumulates an internal buffer, which it works on in parallel. It then yields the documents in order, one-by-one.

- batch_size: number of docs to process per thread
- n_threads: number threads to use (-1 is the default that let's SpaCy decide)
- disable: Names of pipeline components to disable to speed up text processing.
                                    

In [ ]:
from spacy.pipeline import Pipe

In [ ]:
# create a dataframe with a subset of the data, mentioning the word immune
immune_df = df[df.text.str.contains('immune')].text

# print the count of matches
print('Lines with the term immune: {}\n'.format(len(immune_df)))

# view the first five section names
for line in immune_df.head(2):
    print(line)
    print()

In [ ]:
%%time

for doc in nlp.pipe(immune_df.head(10)): # includes ['parser','tagger','ner']
    if 'immune' in doc.text:
        print(doc)
        print()

### SpaCy - Tips for faster processing

You can substantially speed up the time it takes SpaCy to read a document by disabling components of the NLP that are not necessary for a given task.

- Disable options: **parser, tagger, ner**

In [ ]:
%%time

# processing occurs ~75x faster by disabling pipeline components
for doc in nlp.pipe(immune_df.head(10), disable=['parser','tagger','ner']):
    if 'immune' in doc.text:
        print(doc)
        print()

### Exercise
1. print all the distinct entities tagged with 'CARDINAL'
2. print all the distinct entities tagged with 'PERSON'
3. print all the distinct entities tagged with 'GPE'

For all exercises:
- use a batch size of 100
- disable the parser and tagger (ner is needed to add the tags)

In [ ]:
%%time

# print all the distinct entities tagged as a CARDINAL
# search in immune_df.head(200)


In [ ]:
%%time

# print all the distinct entities tagged as an organization (ORG)
# search in immune_df.head(500)


In [ ]:
%%time

# print all the distinct entities tagged as a geopolitical entity (GPE)
# search in immune_df.head(1000)


##### Collections - DefaultDict

Usually, a Python dictionary throws a KeyError if you try to get an item with a key that is not currently in the dictionary. The defaultdict in contrast will simply create any items that you try to access (provided of course they do not exist yet). To create such a "default" item, it calls the function object that you pass in the constructor (more precisely, it's an arbitrary "callable" object, which includes function and type objects). For the first example, default items are created using int(), which will return the integer object 0. For the second example, default items are created using list(), which returns a new empty list object.

In [ ]:
from collections import defaultdict

In [ ]:
pubmed_sentence = """PubMed Description: 
PubMed comprises more than 28 million citations for biomedical literature from MEDLINE, life science journals, and online books.
Citations may include links to full-text content from PubMed Central and publisher web sites.""".strip()

example_doc = nlp(pubmed_sentence)

In [ ]:
# WRONG APPROACH - KeyError!

# try to create a word count dict with new keys
d = {}
for word in example_doc:
    d[word] += 1  # cannot add if the key does not exist

print(d)

In [ ]:
d = defaultdict(int)  # define the type of data the dict stores
for word in example_doc:
    d[word.text] += 1  # can add to unassigned keys

print(d)

##### Collections - Counter

A Counter is a dict subclass for counting hashable objects. It is an unordered collection where elements are stored as dictionary keys and their counts are stored as dictionary values. Counts are allowed to be any integer value including zero or negative counts. The Counter class is similar to bags or multisets in other languages.

SOURCE: https://docs.python.org/2/library/collections.html#collections.Counter

In [ ]:
from collections import Counter

In [ ]:
# count the number of times each CARDINAL appears
print(Counter(d))

##### Iterrtools - combinations

"The itertools module standardizes a core set of fast, memory efficient tools that are useful by themselves or in combination. Together, they form an “iterator algebra” making it possible to construct specialized tools succinctly and efficiently in pure Python.

**Combinations**
- Return r length subsequences of elements from the input iterable.
- Combinations are emitted in lexicographic sort order. So, if the input iterable is sorted, the combination tuples will be produced in sorted order.
- Elements are treated as unique based on their position, not on their value. So if the input elements are unique, there will be no repeat values in each combination.

SOURCE: https://docs.python.org/3.4/library/itertools.html

In [ ]:
from itertools import combinations

In [ ]:
terms = ['PubMed','Medline','Citation','Biomedical']
for combo in combinations(terms, 2):
    print(combo)

##### List Comprehensions

In [ ]:
# traditional iteration

terms_subset = []
for term in terms:
    if 'med' in term.lower():
        terms_subset.append(term)

terms_subset

In [ ]:
# list comprehension

#   return value   iteration           conditional
#[  term           for term in terms   if 'med' in term.lower()]

[term for term in terms if 'med' in term.lower()]

##### Sorted

sorted(iterable, key=None, reverse=False)

- Return a new sorted list from the items in iterable.
- Has two optional arguments which must be specified as keyword arguments.
- key specifies a function of one argument that is used to extract a comparison key from each list element: key=str.lower. The default value is None (compare the elements directly).
- reverse is a boolean value. If set to True, then the list elements are sorted as if each comparison were reversed.

SOURCE: https://docs.python.org/3/library/functions.html#sorted

In [ ]:
articles =[('article2', 3),('article3', 2),('article1', 1)]

In [ ]:
sorted(articles)

In [ ]:
sorted(articles, key=lambda x:x[1])

In [ ]:
sorted(articles, key=lambda x:x[1], reverse=True)

In [ ]:
# sort based on the last character of the first term
sorted(articles, key=lambda x:x[0][-1])

### Exercise
1. Count how many time each individual entity appears
2. Create a mapping that keeps track of every combination of entities pairs that appear in the same sentence
3. Count how many times each entity combo appears
4. Print the entity combos (using sorted) in descending order

In [ ]:
%%time

# create a defaultdict(int) called entity_relations
entity_relations = 

# create an empty list called counter_entities 
counter_entities = 

# during testing set .head() to a smaller number such as .head(1000) 
for doc in nlp.pipe(immune_df.head(10000), disable=['parser','tagger', 'ner']):

    # store the token.text for all the tokens containing the letters 'toxic' (i.e. 'toxic' in term)
    # use a list comprehension
    entities = 

    # add the tokens from the current doc to counter_entities (use += to add the token.text)
    counter_entities 
    
    # create combinations of two terms each time multiple 'toxic' words appear
    # increment the count in entity_relations defaultdict each time a combo is repeated
    for combo in combinations(entities, 2):
        entity_relations[combo] += 1

In [ ]:
print(Counter(counter_entities))

In [ ]:
# view the entity pairs in descending order
sorted(entity_relations.items(), key=lambda x: x[1], reverse=True)

### Identify Relevant Text (Rule-based Matching)

Finding sequences of tokens based on their texts and linguistic annotations, similar to regular expressions. We will use this to filter and extract relevant text.

In [ ]:
rule_basesd_matching_url = 'https://spacy.io/usage/linguistic-features#rule-based-matching'
iframe = '<iframe src={} width=1000 height=700></iframe>'.format(rule_basesd_matching_url)
HTML(iframe)

In [ ]:
# The Matcher identifies text from rules we specify
from spacy.matcher import Matcher

In [ ]:
# create a function to specify what to do with the matching text

def collect_sents(matcher, doc, i, matches):
    """  collect and transform matching text

    :param matcher: Matcher object
    :param doc: is the full document to search for text patterns
    :param i: is the index of the text matches
    :param matches: matches found in the text
    """
    
    match_id, start, end = matches[i]  # indices of matched term
    span = doc[start:end]              # extract matched term
    
    print('span: {} | start_ind:{:5} | end_ind:{:5} | id:{}'.format(
        span, start, end, match_id))

In [ ]:
%%time

# set a pattern of text to collect
# find all mentions of the word fees 
pattern = [{'LOWER':'disease'}] # LOWER coverts words to lowercase before matching

# instantiate matcher
matcher = Matcher(nlp.vocab)

# add pattern to the matcher (one matcher can look for many unique patterns)
# provice a pattern name, function to apply to matches, pattern to identify
matcher.add('disease', collect_sents, pattern)

# pass the doc to the matcher to run the collect_sents function
for doc in nlp.pipe(immune_df.head(100), disable=['parser','tagger','ner']): 
    matcher(doc)

In [ ]:
# change the function to print the sentence of the matched term (span)

def collect_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start:end]
    print('SPAN: {}'.format(span))

    # span.sent provides the sentence that contains the span
    print('SENT: {}'.format(span.sent))
    print()


# update the pattern to look for any noun preceeding the term 'fees'
pattern = [{'POS': 'NOUN', 'OP': '+'},{'LOWER':'disease'}]
matcher = Matcher(nlp.vocab)  # reinstantiate the matcher to remove previous patterns
matcher.add('disease', collect_sents, pattern)

for doc in nlp.pipe(immune_df.head(100), disable=['parser','ner']): # enable pos tagger
    matcher(doc)

In [ ]:
# change the function to collect sentences

def collect_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start:end]
        
    # update matched data collections
    matched_sents.append(span.sent)


matched_sents = []  # container for sentences
pattern = [{'POS': 'NOUN', 'OP': '+'},{'LOWER':'disease'}]
matcher = Matcher(nlp.vocab)
matcher.add('disease', collect_sents, pattern)

for doc in nlp.pipe(immune_df.head(100), disable=['ner']): # enable parser to collect sents
    matcher(doc)

In [ ]:
# review matches
set(matched_sents)

In [ ]:
# change the function to count matches using defaultdict

def collect_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start:end]
    
    # update matched data collections
    ent_count[span.text] += 1  # defaultdict keys must use span.text not span!


ent_count = defaultdict(int)
pattern = [{'LOWER':'disease'}]
matcher = Matcher(nlp.vocab)
matcher.add('disease', collect_sents, pattern)

for doc in nlp.pipe(immune_df.head(100), disable=['pos','ner']): # enable parser to collect sents
    matcher(doc)

ent_count

In [ ]:
%%time

# update the pattern to look for a noun describing the fee

ent_count = defaultdict(int)

# change OP to 1 to only get a single term to the left
pattern = [{'POS': 'NOUN', 'OP': '1'},{'LOWER':'disease'}]
matcher = Matcher(nlp.vocab)
matcher.add('disease', collect_sents, pattern)

for doc in nlp.pipe(immune_df.head(1000), disable=['ner']): # enable parser to collect sents
    matcher(doc)

In [ ]:
ent_count

## Advanced SpaCy (Bonus Material)

##### Text Matching: Avoid Multiple Term Matches

When using rule-based matching, SpaCy may match the same term multiple times if it is part of different n-term pairs with one term contained in another. For instance, 'integration services' in 'system integration services.'

To avoid matching these terms multiple times, we can add to the collect_sents function to check if each term is contained in the previous term

In [ ]:
def collect_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start:end]
    sent = span.sent
    
    # lemmatize the matched spans
    entity = span.lemma_.lower()
            
    # explicity add the first entity without checking if it matches other terms
    # as there is no previous span to check    
    if i == 0:
        ent_count[entity] += 1
        ent_sents[entity].append(sent)
        matched_sents.append(sent)
        return

    # get the span, entity, and sentence from the previous match
    # if more than one match exist
    last_match_id, last_start, last_end = matches[i-1]
    last_span = doc[last_start : last_end]
    last_entity = last_span.text.lower()
    last_sent = last_span.sent

    # to avoid adding duplicates when one term is contained in another 
    # (e.g. 'integration services' in 'system integration services')
    # make sure new spans are unique
    distinct_entity = (entity not in last_entity) or (sent != last_sent)
    not_duplicate_entity = (entity != last_entity) or (sent != last_sent)
    
    # update collections for unique data
    if distinct_entity and not_duplicate_entity:
        ent_count[entity] += 1
        ent_sents[entity].append(sent)
        matched_sents.append(sent)

##### Multiple Patterns

SpaCy matchers can use multiple patterns. Each pattern can be added to the Matcher individually with match.add and can use their own collect_sents function. Or use *patterns to add multiple patterns to the matcher at once.

In [ ]:
matched_sents = []
ent_sents  = defaultdict(list)
ent_count = defaultdict(int)

# multiple patterns
pattern = [[{'POS': 'NOUN', 'OP': '+'},{'LOWER': 'disease'}]
           , [{'POS': 'NOUN', 'OP': '+'},{'LOWER': 'disorder'}]]
matcher = Matcher(nlp.vocab)

# *patterns to add multiple patterns with the same collect_sents function
matcher.add('disease_disorder', collect_sents, *pattern)

for doc in nlp.pipe(immune_df.head(500), disable=['ner']):
    matches = matcher(doc) 

In [ ]:
ent_count

In [ ]:
ent_sents

### Subject Verb Object (S,V,O) Extraction

SOURCES: 
- http://textacy.readthedocs.io/en/latest/_modules/textacy/extract.html#subject_verb_object_triples
- http://textacy.readthedocs.io/en/latest/_modules/textacy/spacy_utils.html#get_main_verbs_of_sent
- https://github.com/chartbeat-labs/textacy/blob/master/textacy/constants.py

In [ ]:
from numpy import nanmin, nanmax, zeros, NaN
from itertools import takewhile
from spacy.parts_of_speech import CONJ, DET, NOUN, VERB
from spacy.tokens.span import Span as SpacySpan

NUMERIC_NE_TYPES = {'ORDINAL', 'CARDINAL', 'MONEY', 'QUANTITY', 'PERCENT', 'TIME', 'DATE'}
SUBJ_DEPS = {'agent', 'csubj', 'csubjpass', 'expl', 'nsubj', 'nsubjpass'}
OBJ_DEPS = {'attr', 'dobj', 'dative', 'oprd'}
AUX_DEPS = {'aux', 'auxpass', 'neg'}

def subject_verb_object_triples(doc):
    """
    Extract an ordered sequence of subject-verb-object (SVO) triples from a
    spacy-parsed doc. Note that this only works for SVO languages.

    Args:
        doc (``textacy.Doc`` or ``spacy.Doc`` or ``spacy.Span``)

    Yields:
        Tuple[``spacy.Span``, ``spacy.Span``, ``spacy.Span``]: the next 3-tuple
            of spans from ``doc`` representing a (subject, verb, object) triple,
            in order of appearance
    """
    # TODO: What to do about questions, where it may be VSO instead of SVO?
    # TODO: What about non-adjacent verb negations?
    # TODO: What about object (noun) negations?
    if isinstance(doc, SpacySpan):
        sents = [doc]
    else:  # textacy.Doc or spacy.Doc
        sents = doc.sents

    for sent in sents:
        start_i = sent[0].i

        verbs = get_main_verbs_of_sent(sent)
        for verb in verbs:
            subjs = get_subjects_of_verb(verb)
            if not subjs:
                continue
            objs = get_objects_of_verb(verb)
            if not objs:
                continue

            # add adjacent auxiliaries to verbs, for context
            # and add compounds to compound nouns
            verb_span = get_span_for_verb_auxiliaries(verb)
            verb = sent[verb_span[0] - start_i: verb_span[1] - start_i + 1]
            for subj in subjs:
                subj = sent[get_span_for_compound_noun(subj)[0] - start_i: subj.i - start_i + 1]
                for obj in objs:
                    if obj.pos == NOUN:
                        span = get_span_for_compound_noun(obj)
                    elif obj.pos == VERB:
                        span = get_span_for_verb_auxiliaries(obj)
                    else:
                        span = (obj.i, obj.i)
                    obj = sent[span[0] - start_i: span[1] - start_i + 1]

                    yield (subj, verb, obj)

def get_main_verbs_of_sent(sent):
    """Return the main (non-auxiliary) verbs in a sentence."""
    return [tok for tok in sent
            if tok.pos == VERB and tok.dep_ not in {'aux', 'auxpass'}]

def get_subjects_of_verb(verb):
    """Return all subjects of a verb according to the dependency parse."""
    subjs = [tok for tok in verb.lefts
             if tok.dep_ in SUBJ_DEPS]
    # get additional conjunct subjects
    subjs.extend(tok for subj in subjs for tok in _get_conjuncts(subj))
    return subjs

def get_objects_of_verb(verb):
    """
    Return all objects of a verb according to the dependency parse,
    including open clausal complements.
    """
    objs = [tok for tok in verb.rights
            if tok.dep_ in OBJ_DEPS]
    # get open clausal complements (xcomp)
    objs.extend(tok for tok in verb.rights
                if tok.dep_ == 'xcomp')
    # get additional conjunct objects
    objs.extend(tok for obj in objs for tok in _get_conjuncts(obj))
    return objs

def _get_conjuncts(tok):
    """
    Return conjunct dependents of the leftmost conjunct in a coordinated phrase,
    e.g. "Burton, [Dan], and [Josh] ...".
    """
    return [right for right in tok.rights
            if right.dep_ == 'conj']

def get_span_for_verb_auxiliaries(verb):
    """
    Return document indexes spanning all (adjacent) tokens
    around a verb that are auxiliary verbs or negations.
    """
    min_i = verb.i - sum(1 for _ in takewhile(lambda x: x.dep_ in AUX_DEPS,
                                              reversed(list(verb.lefts))))
    max_i = verb.i + sum(1 for _ in takewhile(lambda x: x.dep_ in AUX_DEPS,
                                              verb.rights))
    return (min_i, max_i)

def get_span_for_compound_noun(noun):
    """
    Return document indexes spanning all (adjacent) tokens
    in a compound noun.
    """
    min_i = noun.i - sum(1 for _ in takewhile(lambda x: x.dep_ == 'compound',
                                              reversed(list(noun.lefts))))
    return (min_i, noun.i)

In [ ]:
for doc in nlp.pipe(immune_df.head(50), disable=['pos','ner']): # enable parser to collect sents
    
    # collect triples
    triples = [(s,v,o) for s,v,o in subject_verb_object_triples(doc)]
    
    if triples:
        print(triples)